## Importing Libraries

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import dask.array as da
from skimage.transform import resize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install keras-video-generators

In [ ]:
import os
import glob
import keras

from tensorflow.keras.utils import Sequence

In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
import os

In [ ]:
import tensorflow
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling3D, Conv3D
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from keras.regularizers import l2

In [ ]:
# %pwd
# %ls drive/MyDrive/Deep\ Learning\ Project/frames
# f.head()

# from google.colab import drive
# drive.mount('/content/drive')

# Extracting Frames

The below function should extract exactly 4 frames from the video between the 1st and the 2nd second.

In [ ]:
def to_frames_2(shot):
  count = 0

  #Initialising the X and Y variables. The mapping dataframe can be converted for one-hot encoded for image labelling.
  mapping = pd.DataFrame(columns = ['frames', 'labels', 'video'])

  #extracting frames from each image in the dataset
  for j in range(len(shot)):
    subf = f[f['shots'] == shot[j]].reset_index()
    image_data = []
    for i in range(len(subf)):
      videoFile = subf['links'][i]
      # print(videoFile)
      # img2 = []
      cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
      frameRate = cap.get(5) #frame rate
      x=1
      count_frames = 0
      while(cap.isOpened()):
          frameId = cap.get(1) #current frame number
          ret, frame = cap.read()
          if (ret != True):
              break
          # print(frameId)
          if ((frameId > frameRate*1) & (frameId < frameRate*2) & (frameId % 3 == 0) & (frameRate >= 12)):
              count_frames+=1
              if (count_frames <= 4):
                filename = subf['save'][i] + "frame%d.jpg" % count
                # print(filename)
                cv2.imwrite(filename, frame)
                mapping = mapping.append({
                    'frames' : "frame%d" % count,
                    'labels' : subf['shots'][i],
                    'video' : count_frames
                }, ignore_index = True)
                img = plt.imread('' + filename)
                image_data.append(img)
                # img2.append(img)
                count+=1
              else:
                continue
      # print(count_frames)
      cap.release()
      # img3 = np.array(img2).reshape()
    image_data = np.array(image_data)
    np.save(subf['save'][0] + 'x_new_' + shot[j] + '.npy', image_data)
  print ("Done!")
  return mapping

## Extracting Ground Shots

In [ ]:
#string paths from where the videos will be read and frames will be saved.
directory = '/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/vids/'
saving_dir = '/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/'


#shots = ['backhand', 'backhand2hands', 'backhand_slice', 'backhand_volley', 'flat_service', 'forehand_flat', 'forehand_openstands', 'forehand_slice', 'forehand_volley', 'kick_service', 'slice_service', 'smash']

shots = ['backhand2hands','forehand_flat']

#creating a dataframe to store all the paths
f = pd.DataFrame(columns = ['shots', 'links', 'save'])

for i in shots:
  paths = directory + i + "/"
  saving_path = saving_dir
  for filename in os.listdir(paths):
      f = f.append({
          'shots': i,
          'links': (os.path.join(paths,filename)),
          'save' : (os.path.join(saving_path)),
      }, ignore_index = True)

In [ ]:
%%time
mapping = to_frames_2(['backhand2hands','forehand_flat'])

Done!
CPU times: user 1min 21s, sys: 4.9 s, total: 1min 25s
Wall time: 1min 27s


In [ ]:
mapping['groundshots'] = 1
mapping['service'] = 0

In [ ]:
mapping

,frames,labels,video,groundshots,service
0,frame0,backhand2hands,1,1,0
1,frame1,backhand2hands,2,1,0
2,frame2,backhand2hands,3,1,0
3,frame3,backhand2hands,4,1,0
4,frame4,backhand2hands,1,1,0
...,...,...,...,...,...
1303,frame1303,forehand_flat,4,1,0
1304,frame1304,forehand_flat,1,1,0
1305,frame1305,forehand_flat,2,1,0
1306,frame1306,forehand_flat,3,1,0


## Extracting Service Shots

In [ ]:
#string paths from where the videos will be read and frames will be saved.
directory = '/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/vids/'
saving_dir = '/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/'


#shots = ['backhand', 'backhand2hands', 'backhand_slice', 'backhand_volley', 'flat_service', 'forehand_flat', 'forehand_openstands', 'forehand_slice', 'forehand_volley', 'kick_service', 'slice_service', 'smash']

shots = ['flat_service', 'kick_service', 'slice_service']

#creating a dataframe to store all the paths
f = pd.DataFrame(columns = ['shots', 'links', 'save'])

for i in shots:
  paths = directory + i + "/"
  saving_path = saving_dir
  for filename in os.listdir(paths):
      f = f.append({
          'shots': i,
          'links': (os.path.join(paths,filename)),
          'save' : (os.path.join(saving_path)),
      }, ignore_index = True)

In [ ]:
mapping2 = to_frames_2(['flat_service', 'kick_service', 'slice_service'])

Done!


In [ ]:
mapping2['groundshots'] = 0
mapping2['service'] = 1

## Merging and combining dataset for training

In [ ]:
y_ground = np.array(mapping[['groundshots', 'service']])

In [ ]:
y_service = np.array(mapping2[['groundshots', 'service']])

In [ ]:
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/y_ground.npy", y_ground)

In [ ]:
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/y_service.npy", y_service)

In [ ]:
x1 = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/x_new_backhand2hands.npy")
x2 = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/x_new_forehand_flat.npy")

x_new = np.append(x1, x2, axis=0)

np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/x_new.npy", x_new)

x_1_1 = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/x_new_flat_service.npy")
x_2_2 = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/x_new_kick_service.npy")
x_3_3 = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/x_new_slice_service.npy")

x_new = np.array([])
x_new = np.append(x_1_1, x_2_2, axis=0)
del x_1_1, x_2_2
x_new = np.append(x_new,x_3_3, axis=0)
del x_3_3

np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/x_new.npy", x_new)

## Reloading and combining the data

In [ ]:
x_ground = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/x_new.npy")
x_service = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/x_new.npy")
y_ground = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/groundshots-frames2/y_ground.npy")
y_service = np.load("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/service-frames2/y_service.npy")

In [ ]:
x_new = np.append(x_ground, x_service, axis=0)
x_new.shape

(3288, 480, 640, 3)

In [ ]:
y_new = np.append(y_ground, y_service, axis=0)
y_new.shape

(3288, 2)

In [ ]:
x_train = x_new[0:916]
x_test = x_new[916:1308]
x_train = np.append(x_train, x_new[1308:2696], axis=0)
x_test = np.append(x_test, x_new[2696:], axis=0)

y_train = y_new[0:916]
y_test = y_new[916:1308]
y_train = np.append(y_train, y_new[1308:2696], axis=0)
y_test = np.append(y_test, y_new[2696:], axis=0)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2304, 480, 640, 3)
(984, 480, 640, 3)
(2304, 2)
(984, 2)


In [ ]:
del x_new, y_new, y_ground, y_service, x_ground, x_service

## resizing Images

In [ ]:
x_train_resized = []
x_test_resized = []

for i in x_train:
    x_train_resized.append(resize(i, (120, 160)))

for i in x_test:
    x_test_resized.append(resize(i, (120, 160)))

x_train_resized = np.array(x_train_resized)
x_test_resized = np.array(x_test_resized)

print(x_train_resized.shape, x_test_resized.shape)

(2304, 120, 160, 3) (984, 120, 160, 3)


In [ ]:
del x_train, x_test

## Making the 3d array into 4d

In [ ]:
x_train_vids = x_train_resized[0:4][np.newaxis,...]

for i in range(4,x_train_resized.shape[0],4):
    sample = x_train_resized[i:i+4]
    sample = sample[np.newaxis,...]
    x_train_vids = np.append(x_train_vids,sample,axis=0)

In [ ]:
x_train_resized.shape

(2304, 120, 160, 3)

In [ ]:
x_train_vids.shape

(576, 4, 120, 160, 3)

In [ ]:
x_test_vids = x_test_resized[0:4][np.newaxis,...]

for i in range(4,x_test_resized.shape[0],4):
    sample = x_test_resized[i:i+4]
    sample = sample[np.newaxis,...]
    x_test_vids = np.append(x_test_vids,sample,axis=0)

In [ ]:
x_test_vids.shape

In [ ]:
y_train_vids = y_train[::4]
y_train_vids.shape

y_test_vids = y_test[::4]
y_train_vids.shape

(576, 2)

In [ ]:
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/x_train_vids.npy", x_train_vids)
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/x_test_vids.npy", x_test_vids)
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/y_train.npy", y_train)
np.save("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Deep Learning/Data For Project/Frames_binary/y_test.npy", y_test)